In [1]:
from google.colab import drive
from nltk.corpus import stopwords
import nltk
from collections import defaultdict
import re
from nltk.stem.porter import PorterStemmer
import heapq
import json
import gc
porter = PorterStemmer()

nltk.download('stopwords')
stop_words = stopwords.words('english')

def remove_symbols(line):
    return re.sub('[^A-Za-z0-9\s]+', '', line).lower()

"""
we are using this to change list into set while dumping json into file
"""
class SetEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, set):
      return list(obj)
    return json.JSONEncoder.default(self, obj)

import csv
import sys
csv.field_size_limit(sys.maxsize)

BLOCK_SIZE = 100000

def bsbi():
  freq_dict = defaultdict(set)
  with open('gutenberg_data.csv') as f:
    next(f) # just skipping first line(column headers)
    csv_file = csv.reader(f)
    total_files = 0
    i = 0
    current_block = 0
    for line in csv_file:
      title, author, link, id, bookshelf, text = line
      
      for word in text.split():
        word = remove_symbols(word)
        if word and word not in stop_words:
          word = porter.stem(word)
          if word not in freq_dict:
             
            current_block += 1
          
          if not freq_dict[word].__contains__(id):
            freq_dict[word].add(id)
            current_block += 1
        if current_block >= BLOCK_SIZE:
           
          sorted_list = sorted(freq_dict.items(), key=lambda _: _[0])  
          with open(f'./IROUTPUTS_v2/OP{total_files}.txt', 'w') as  f:
            # 
            for word_id, docs in sorted_list:
              f.write(word_id)
              for doc_id in docs:
                f.write(f' {doc_id}')
              f.write('\n')
          current_block = 0
          freq_dict.clear()
          total_files += 1
          print(i, ' rows done')
          # if total_files == 2:
          #   return
      
    sorted_list = sorted(freq_dict.items(), key=lambda _: _[0]) # sorting by word_id
 
    if len(sorted_list) > 0:
      with open(f'./IROUTPUTS_v2/OP{total_files}.txt', 'w') as  f:
      
        for word_id, docs in sorted_list:
          f.write(word_id)
          for doc_id in docs:
            f.write(f' {doc_id}')
          f.write('\n')
      current_block = 0
      freq_dict.clear()
      total_files += 1
bsbi()

file_names = [f'./IROUTPUTS/OP{i}.txt' for i in range(947)]
file_pointers = [open(i) for i in file_names]

def decorated_file(f, key):
  for line in f:
    yield (key(line), line)

files = map(open, file_names)
outfile = open('./IROUTPUTS_v2/merged.txt', 'w')

def key_fn(line):
    return line.split(' ', 2)[0]

prev = ''
for line in heapq.merge(*[decorated_file(f, key_fn) for f in files]):
   if prev != line[0]:
     outfile.write(f'\n{line[1].strip()}')
    prev = line[0]
   else:
     outfile.write(f' {line[1][len(line[0]):].strip()}')
for i in file_pointers:
  i.close()
outfile.close()

for i in file_pointers:
  i.close()
outfile.close()

IndentationError: ignored